In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print("라이브러리 설치 시작...")

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install ultralytics segment-anything segment-anything-hq pillow matplotlib opencv-python --upgrade
!pip uninstall -y segmentation-models-pytorch

print("라이브러리 설치 완료.")

가벼운 샘모델 다운로드

In [ ]:
!mkdir -p models
!wget -O models/sam_hq_vit_b.pth https://huggingface.co/lkeab/hq-sam/resolve/main/sam_hq_vit_b.pth


✅ 1) 필요 패키지 설치

In [ ]:
!pip install fastapi uvicorn pyngrok streamlit
!pip install ultralytics segment-anything-hq opencv-python Pillow

✅ 2) ngrok 토큰 등록

In [ ]:
!ngrok config add-authtoken ""

✅ 3) model_server.py 생성 (FastAPI + YOLO + SAM + 추천 로직)

In [ ]:
%%writefile model_server.py
# -*- coding: utf-8 -*-
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware

import numpy as np
import cv2
import pandas as pd
import torch
from ultralytics import YOLO
from segment_anything_hq import sam_model_registry, SamPredictor

from PIL import Image
import base64
import io
from collections import Counter


# ============================================================
# 0) FastAPI + CORS
# ============================================================
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)



# ============================================================
# 1) 경로 설정
# ============================================================
#YOLO_MODEL_PATH = "/content/drive/MyDrive/최종프로젝트 /4조 파이널 프로젝트/학습에 필요한 파일/손톱영역_세그멘테이션_모델/2.욜로11s/best.pt"
YOLO_MODEL_PATH = "../02_nail_seg_yolo/best.pt"

SAM_CHECKPOINT_PATH = "models/sam_hq_vit_b.pth"
#RECOMMEND_CSV = "/content/drive/MyDrive/최종프로젝트 /4조 파이널 프로젝트/학습에 필요한 파일/네일 모양, 디자인 라벨링csv/nail_design_recommend_table.csv" # 추천 테이블 CSV
RECOMMEND_CSV = "../01_data_text_rule/basic_data/04_group_shape_design_rule.csv"

design_df = pd.read_csv(RECOMMEND_CSV)
design_df["ranking"] = pd.to_numeric(design_df["ranking"], errors="coerce")

device = "cuda" if torch.cuda.is_available() else "cpu"


# ============================================================
# 2) 모델 로드
# ============================================================
print("🚀 Loading YOLO model...")
yolo_model = YOLO(YOLO_MODEL_PATH)

print("🚀 Loading SAM-HQ-B model...")
sam = sam_model_registry["vit_b"](checkpoint=SAM_CHECKPOINT_PATH)
sam.to(device)
predictor = SamPredictor(sam)



# ============================================================
# 3) 추천 Top3
# ============================================================
type_to_group = {
    "long-narrow": "L_N",
    "long-wide": "L_W",
    "mid-narrow": "M_N",
    "mid-wide": "M_W",
    "short-narrow": "S_N",
    "short-wide": "S_W",
}

def get_top3(group_id):
    df = design_df.copy()
    df = df[df["group_id"] == group_id]

    if df.empty:
        return []

    df = df.sort_values("ranking")
    top3 = df.head(3)

    return top3[["shape", "design"]].to_dict(orient="records")



# ============================================================
# 4) 유틸: PIL → base64
# ============================================================
def pil_to_base64(img):
    buffer = io.BytesIO()
    img.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode()



# ============================================================
# 5) 손톱 길이/폭 분석
# ============================================================
def analyze_shape(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return None

    contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])
    widths = [cv2.boundingRect(c)[2] for c in contours]
    width_median = np.median(widths)

    types = []

    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        aspect = h / max(w, 1)

        if aspect < 1.2:
            length = "short"
        elif aspect < 1.6:
            length = "mid"
        else:
            length = "long"

        width_type = "narrow" if w < width_median else "wide"
        types.append(f"{length}-{width_type}")

    final_type = Counter(types).most_common(1)[0][0]
    return final_type



# ============================================================
# 6) Predict API
# ============================================================
@app.post("/predict")
async def predict(file: UploadFile = File(...)):

    # ------------------------------
    # 1) 이미지 로드
    # ------------------------------
    contents = await file.read()
    pil_img = Image.open(io.BytesIO(contents)).convert("RGB")

    # SAM 가속 → 리사이즈 (너무 크면 느림)
    W, H = pil_img.size
    if W > 800:
        scale = 800 / W
        pil_img = pil_img.resize((800, int(H * scale)))

    image = np.array(pil_img)


    # ------------------------------
    # 2) YOLO 탐지
    # ------------------------------
    result = yolo_model.predict(image, conf=0.5, verbose=False)[0]

    if len(result.boxes) == 0:
        return JSONResponse({"error": "손톱을 찾지 못했습니다."}, status_code=400)

    boxes = result.boxes.xyxy.cpu().numpy()

    yolo_vis = cv2.cvtColor(result.plot(), cv2.COLOR_BGR2RGB)
    yolo_pil = Image.fromarray(yolo_vis)


    # ------------------------------
    # 3) SAM segmentation
    # ------------------------------
    predictor.set_image(image)

    masks = []
    for x1, y1, x2, y2 in boxes:
        box = np.array([[x1, y1, x2, y2]])
        mask, _, _ = predictor.predict(box=box, multimask_output=False)
        masks.append(mask[0])

    final_mask = np.any(masks, axis=0).astype(np.uint8) * 255


    # ------------------------------
    # 4) 손톱 길이/폭 타입 분석
    # ------------------------------
    nail_type = analyze_shape(final_mask)
    if nail_type is None:
        return JSONResponse({"error": "마스크 생성 실패"}, status_code=400)

    group_id = type_to_group[nail_type]


    # ------------------------------
    # 5) 디자인 추천 Top3
    # ------------------------------
    top3 = get_top3(group_id)


    # ------------------------------
    # 6) 마스크 시각화 이미지 만들기
    # ------------------------------
    overlay = image.copy()
    overlay[final_mask == 255] = (overlay[final_mask == 255] * 0.6 + np.array([0, 0, 255]) * 0.4).astype(np.uint8)
    mask_pil = Image.fromarray(overlay)


    # ------------------------------
    # 7) JSON 반환
    # ------------------------------
    return {
        "status": "ok",
        "nail_type": nail_type,
        "group_id": group_id,
        "recommendations": top3,
        "yolo_image": pil_to_base64(yolo_pil),
        "mask_image": pil_to_base64(mask_pil)
    }



# ============================================================
# 7) 서버 실행
# ============================================================
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


✅ 5) FastAPI 서버 실행

In [ ]:
!uvicorn model_server:app --host 0.0.0.0 --port 8000 &


✅ 6) ngrok으로 FastAPI 외부주소 생성 초기화

In [ ]:
from pyngrok import ngrok
ngrok.kill()
public_url = ngrok.connect(8000)
public_url
